In [13]:
source_code = """
int main(args) {
    int x = 42;
        /*
    multiline comment
    */
    printf("Hello, World!");
    return 0;
}
"""

In [14]:
tokenize(source_code)

[Token(kind=<TokenKind.KEYWORD: 2>, value=<CKeywords.INT: 'int'>, start=4, end=6),
 Token(kind=<TokenKind.IDENTIFIER: 1>, value='main', start=9, end=12),
 Token(kind=<TokenKind.SYMBOL: 3>, value='(', start=10, end=10),
 Token(kind=<TokenKind.IDENTIFIER: 1>, value='args', start=14, end=17),
 Token(kind=<TokenKind.SYMBOL: 3>, value=')', start=15, end=15),
 Token(kind=<TokenKind.SYMBOL: 3>, value='{', start=17, end=17),
 Token(kind=<TokenKind.KEYWORD: 2>, value=<CKeywords.INT: 'int'>, start=25, end=27),
 Token(kind=<TokenKind.IDENTIFIER: 1>, value='x', start=27, end=27),
 Token(kind=<TokenKind.OPERATOR: 4>, value='=', start=29, end=29),
 Token(kind=<TokenKind.NUMBER: 6>, value='42', start=32, end=33),
 Token(kind=<TokenKind.SYMBOL: 3>, value=';', start=33, end=33),
 Token(kind=<TokenKind.COMMENT: 7>, value='\n    multiline comment\n    ', start=73, end=99),
 Token(kind=<TokenKind.IDENTIFIER: 1>, value='printf', start=84, end=89),
 Token(kind=<TokenKind.SYMBOL: 3>, value='(', start=85, end

In [14]:
bad_code = """
int main() {
    int x = 42;
    /*
    multiline comment
    */
    printf(Hello, World!");
    return 0;
}
"""

In [15]:
tokenize(bad_code)

ValueError: Tokenization Error:
Unexpected token at line 7, column 25:

    multiline comment
    */
    printf(Hello, World!");
                        ^
    return 0;
}
Expected one of whitespace, comment, keywords, (one of {, }, (, ), ;, ,, [, ], ., -> @ create_token), (one of +, -, *, /, %, =, ==, !=, <, >, <=, >=, &&, ||, !, &, |, ^, <<, >> @ create_token), number, string literal, identifier.


In [16]:
tokenizer

Parser(one of whitespace, comment, keywords, (one of {, }, (, ), ;, ,, [, ], ., -> @ create_token), (one of +, -, *, /, %, =, ==, !=, <, >, <=, >=, &&, ||, !, &, |, ^, <<, >> @ create_token), number, string literal, identifier)